<a href="https://colab.research.google.com/github/VladShajdulin/OTUS_project/blob/main/OTUS_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import json
import regex
import torch
from torch import Tensor
from sentence_transformers import SentenceTransformer

In [2]:
torch.manual_seed(142)
path = '/content/drive/MyDrive/Colab Notebooks/OTUS/final_project/articles_links.json'
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else 'cpu'
print(device)

cuda


In [3]:
all_data = pd.read_json(path)
all_data

,title,author,date,imp_ind,views,annot,tags,link
0,"Ученые обнаружили еще одну причину, по которой...",Игорь Байдов,2025-10-10,5.2,21600,Голые землекопы живут почти 40 лет — невероятн...,"[Биология, голые землекопы, ДНК, долголетие]",https://naked-science.ru/article/biology/uchen...
1,"Бактерии создали из себя батарейку, чтобы разр...",Evgenia Vavilova,2025-10-10,4.2,7800,Они использовали содержащееся в почве железо. ...,"[Биология, аккумулятор, бактерии, биопленки]",https://naked-science.ru/article/biology/bacte...
2,Бактерии и добавки пероксида кальция сделают д...,Губкинский университет,2025-10-10,4.8,149,Ученые РГУ нефти и газа (НИУ) имени И.М. Губки...,"[Губкинский университет, арктика, бактерии, ка...",https://naked-science.ru/article/column/bakter...
3,Российские физики предложили искать темную мат...,ФизТех,2025-10-10,5.1,1000,Большой коллектив российских ученых из ведущих...,"[ФизТех, вселенная, квантование гравитации, те...",https://naked-science.ru/article/column/temnuy...
4,Белорусские ученые вырастили кожу из грибов и ...,Unitsky String Technologies Inc.,2025-10-10,4.5,430,Специалисты UST Inc. разработали технологию по...,"[Unitsky String Technologies Inc., биоматериал...",https://naked-science.ru/article/column/beloru...
...,...,...,...,...,...,...,...,...
8267,В Испании нашли свой Стоунхендж,Алиса Гаджиева,2022-08-22,4.3,1300,"На поле, предназначенном для выращивания авока...","[Антропология, археология, древние люди, Испания]",https://naked-science.ru/article/anthropology/...
8268,Астрономы оценили максимально возможное число ...,Сергей Васильев,2022-08-22,5.2,8600,"Ученые подсчитали, что без больших изменений о...","[Астрономия, естественный спутник, Луна, спутн...",https://naked-science.ru/article/astronomy/mak...
8269,Рецепт сознания: могут ли люди и роботы мечтат...,Денис Тулинов,2022-08-22,5.0,7600,В XXI веке вопрос о природе сознания станет од...,"[Технологии, искусственный интеллект, машинное...",https://naked-science.ru/article/hi-tech/retse...
8270,У берегов Западной Африки нашли древний ударны...,Мария Азарова,2022-08-21,6.5,4700,"Ученые допускают, что 66 миллионов лет назад с...","[Геология, астероид, гибель динозавров, диноза...",https://naked-science.ru/article/geology/udarn...


In [4]:
df = all_data[['title', 'link', 'annot']]
df.head(5)

,title,link,annot
0,"Ученые обнаружили еще одну причину, по которой...",https://naked-science.ru/article/biology/uchen...,Голые землекопы живут почти 40 лет — невероятн...
1,"Бактерии создали из себя батарейку, чтобы разр...",https://naked-science.ru/article/biology/bacte...,Они использовали содержащееся в почве железо. ...
2,Бактерии и добавки пероксида кальция сделают д...,https://naked-science.ru/article/column/bakter...,Ученые РГУ нефти и газа (НИУ) имени И.М. Губки...
3,Российские физики предложили искать темную мат...,https://naked-science.ru/article/column/temnuy...,Большой коллектив российских ученых из ведущих...
4,Белорусские ученые вырастили кожу из грибов и ...,https://naked-science.ru/article/column/beloru...,Специалисты UST Inc. разработали технологию по...


In [5]:
def clean_text(text, pattern=r'[^\w\p{P}]+'):
  pattern = regex.compile(pattern)
  text = pattern.sub(' ', text)

  return regex.sub(r'\s+', ' ', text).strip()

In [6]:
texts = [clean_text(t) for t in df['annot']]
retrieve = [f'passage: {t}' for t in texts]
texts[:5]

['Голые землекопы живут почти 40 лет — невероятный срок для грызуна размером с мышь. Недавнее открытие китайских ученых показало, что долголетие этим животным обеспечивает необычная версия одного белка, который не просто защищает организм, а буквально чинит его изнутри — ускоряет восстановление ДНК. Новые данные могут помочь продлить и человеческую жизнь.',
 'Они использовали содержащееся в почве железо. Днем бактерии захватывали фотоны, ночью — боролись с мешающими им жить соединениями.',
 'Ученые РГУ нефти и газа (НИУ) имени И.М. Губкина разработали новый метод обезвреживания нефтесодержащих отходов в Арктической зоне России. Он основан на комбинированном использовании бактериальных препаратов с добавками пероксида кальция, источника кислорода при обезвреживании отходов с использованием геотубных технологий. Технология успешно прошла полевые испытания.',
 'Большой коллектив российских ученых из ведущих физических институтов Москвы, Нижнего Новгорода, Сарова и Санкт-Петербурга предста

In [7]:
model = SentenceTransformer('d0rj/e5-base-en-ru', device=device)
embeddings = model.encode(retrieve,
                     convert_to_tensor=True,
                     show_progress_bar=True,
                     device=device,
                     normalize_embeddings=True)
torch.save(embeddings, '/content/drive/MyDrive/Colab Notebooks/OTUS/final_project/embeddings.pt')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/259 [00:00<?, ?it/s]

In [10]:
embeddings = torch.load('/content/drive/MyDrive/Colab Notebooks/OTUS/final_project/embeddings.pt')
embeddings.shape

torch.Size([8272, 768])

In [8]:
query = 'Мне интересны статьи про магнитное поле нейтронных звезд'
query = f'query: {query}'

In [ ]:
import torch.nn.functional as F
from torch import Tensor

embeddings = F.normalize(embeddings, p=2, dim=1)

In [ ]:
scores = embeddings[:1] @ embeddings[1:].T

In [ ]:
top = 20
raiting = torch.argsort(scores, descending=True)
raiting = raiting[0, :top].tolist()

In [ ]:
df.iloc[raiting].head(5)

,title,link,annot
0,"Наблюдения показали, что у звезд-магнетаров не...",https://naked-science.ru/article/astronomy/u-m...,Некоторые нейтронные звезды обладают экстремал...
828,"Астрономы нашли странную звезду, готовую превр...",https://naked-science.ru/article/astronomy/str...,Некоторые нейтронные звезды обладают колоссаль...
134,Ослабление магнитного поля планеты посчитали п...,https://naked-science.ru/article/geology/oslab...,Более полумиллиарда лет назад магнитное поле З...
419,Солнце оказалось «белой вороной» среди себе по...,https://naked-science.ru/article/astronomy/com...,При сравнении десятков солнцеподобных звезд вы...
486,"Астрономы поняли, как происходит ускорение дже...",https://naked-science.ru/article/astronomy/ast...,"Узкие потоки частиц, разогнанных почти до скор..."


In [ ]:
from sentence_transformers import CrossEncoder

reranker_model = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1')

In [ ]:
documents = [texts[i] for i in raiting]
documents[:5]

['Некоторые нейтронные звезды обладают экстремально мощным магнитным полем. Судя по поляризации излучения одного из таких магнетаров, это поле настолько сильно, что вызывает конденсацию атмосферы и ее переход в твердую фазу.',
 'Некоторые нейтронные звезды обладают колоссальным магнитным полем. До сих пор не ясно, откуда они возникают. Возможно, из необычных светил, подобных участнице далекой системы HD 45166. Новые наблюдения показали, что она может быть первым известным нам примером звезд — предшественниц магнетаров.',
 'Более полумиллиарда лет назад магнитное поле Земли по неизвестной причине ослабело в десятки раз и оставалось таким десятки миллионов лет.',
 'При сравнении десятков солнцеподобных звезд выяснилось, что наше светило разительно отличается почти от всех остальных характером магнитной активности: на нем образуется гораздо больше ярких областей, чем темных пятен. У большинства желтых карликов все наоборот.',
 'Узкие потоки частиц, разогнанных почти до скорости света, ярк

In [ ]:
results = reranker_model.rank(query, documents)
results

[{'corpus_id': 1, 'score': np.float32(1.3000908)},
 {'corpus_id': 0, 'score': np.float32(1.2627608)},
 {'corpus_id': 10, 'score': np.float32(-1.2610025)},
 {'corpus_id': 11, 'score': np.float32(-3.678667)},
 {'corpus_id': 6, 'score': np.float32(-4.948364)},
 {'corpus_id': 5, 'score': np.float32(-4.9582133)},
 {'corpus_id': 14, 'score': np.float32(-4.9750667)},
 {'corpus_id': 16, 'score': np.float32(-5.2497926)},
 {'corpus_id': 3, 'score': np.float32(-5.27209)},
 {'corpus_id': 15, 'score': np.float32(-5.2790103)},
 {'corpus_id': 4, 'score': np.float32(-5.3225794)},
 {'corpus_id': 9, 'score': np.float32(-5.462229)},
 {'corpus_id': 2, 'score': np.float32(-6.0226727)},
 {'corpus_id': 19, 'score': np.float32(-6.02925)},
 {'corpus_id': 12, 'score': np.float32(-6.07647)},
 {'corpus_id': 13, 'score': np.float32(-6.2298894)},
 {'corpus_id': 17, 'score': np.float32(-6.476934)},
 {'corpus_id': 18, 'score': np.float32(-7.735063)},
 {'corpus_id': 8, 'score': np.float32(-8.35464)},
 {'corpus_id': 7,

In [ ]:
ranks = [r['corpus_id'] for r in results]
new_rating = [raiting[i] for i in ranks]
df.iloc[new_rating].head(5)

,title,link,annot
828,"Астрономы нашли странную звезду, готовую превр...",https://naked-science.ru/article/astronomy/str...,Некоторые нейтронные звезды обладают колоссаль...
0,"Наблюдения показали, что у звезд-магнетаров не...",https://naked-science.ru/article/astronomy/u-m...,Некоторые нейтронные звезды обладают экстремал...
357,Необычная сверхновая оказалась взрывом обычног...,https://naked-science.ru/article/column/neob-k...,"Массивные звезды, рождающиеся с массами 9-25 с..."
395,На далекой звезде поднимаются волны высотой бо...,https://naked-science.ru/article/astronomy/na-...,Одна из звезд в системе MACHO 80.7443.1718 сил...
443,"Астрономы выяснили, какие шаровые скопления ча...",https://naked-science.ru/article/astronomy/eje...,В последние годы в нашей Галактике и ее окрест...
